# 第八章 聊天机器人

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
        <li>
        <span><a href="#二身份与上下文构建" data-toc-modified-id="二、身份与上下文构建">二、身份与上下文构建</a></span></li><li>
        <span><a href="#三订餐机器人" data-toc-modified-id="三、订餐机器人">三、订餐机器人</a></span>
        <ul class="toc-item">
            <li><span><a href="#31-创建json摘要" data-toc-modified-id="3.1 创建json摘要">3.1 创建json摘要</a></span></li>
        </ul>
        </li>
    </ul>
</div>

## 一、引言

使用一个大型语言模型的一个令人兴奋的事情是，我们可以用它来构建一个定制的聊天机器人 (Chatbot) ，只需要很少的工作量。在这一节中，我们将探索如何利用聊天的方式，与个性化（或专门针对特定任务或行为的）聊天机器人进行扩展对话。

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

像 ChatGPT 这样的聊天模型实际上是组装成以一系列消息作为输入，并返回一个模型生成的消息作为输出的。这种聊天格式原本的设计目标是简便多轮对话，但我们通过之前的学习可以知道，它对于不会涉及任何对话的**单轮任务**也同样有用。


## 二、身份与上下文构建

接下来，我们将定义两个辅助函数。

第一个方法已经陪伴了您一整个教程，即 ```get_completion``` ，其适用于单轮对话。我们将 Prompt 放入某种类似**用户消息**的对话框中。另一个称为 ```get_completion_from_messages``` ，传入一个消息列表。这些消息可以来自大量不同的**角色** (roles) ，我们会描述一下这些角色。

第一条消息中，我们以系统身份发送<span style="color:red">系统消息 (system message)</span> ，它提供了一个总体的指示。系统消息则有助于设置助手的行为和角色，并作为对话的高级指示。你可以想象它在助手的耳边低语，引导它的回应，而用户不会注意到系统消息。因此，作为用户，如果你曾经使用过 ChatGPT，您可能从来不知道 ChatGPT 的系统消息是什么，这是有意为之的。系统消息的好处是为开发者提供了一种方法，在不让请求本身成为对话的一部分的情况下，引导助手并指导其回应。

在 ChatGPT 网页界面中，您的消息称为用户消息，而 ChatGPT 的消息称为助手消息。但在构建聊天机器人时，在发送了系统消息之后，您的角色可以仅作为<span style="color:red">用户 (user)</span> ；也可以在用户和<span style="color:red">助手 (assistant) </span>之间交替，从而提供对话上下文。

```json
[
  {
    "role": "system",
    "content": "You are an assistant that speaks like Shakespeare."
  },
  {
    "role": "user",
    "content": "tell me a joke"
  },
  {
    "role": "assistant",
    "content": "Why did the chicken cross the road"
  },
  {
    "role": "user",
    "content": "I don't know"
  }
]
```

In [2]:
client = OpenAI(api_key=openai_api_key)

def get_completion(prompt, model="gpt-4o-mini"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为gpt-4o-mini
    '''
    # 调用API
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    # 返回结果
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    # 返回结果
    return response.choices[0].message.content


现在让我们尝试在对话中使用这些消息。我们将使用上面的函数来获取从这些消息中得到的回答，同时，使用更高的温度 (temperature)（越高生成的越多样，更多内容见第七章）。

系统消息说，你是一个说话像莎士比亚的助手。这是我们向助手描述**它应该如何表现的方式**。然后，第一个用户消息是*给我讲个笑话*。接下来以助手身份给出回复是，*为什么鸡会过马路？* 最后发送用户消息是*我不知道*。

In [3]:
# 中文
messages =  [  
{'role':'system', 'content':'你是一个像莎士比亚一样说话的助手。'},    
{'role':'user', 'content':'给我讲个笑话'},   
{'role':'assistant', 'content':'鸡为什么过马路'},   
{'role':'user', 'content':'我不知道'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

鸡为的是到对面去呀！唉，这个笑话虽简单，却引人发思，正如生活中的许多事，关键在于动机与目的。你若有兴致，再来听我说些趣事！


让我们看另一个例子。助手的消息是*你是一个友好的聊天机器人*，第一个用户消息是*嗨，我叫Isa*。我们想要得到第一个用户消息。

In [6]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'Hi, 我是Isa。'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa！👋 很高兴认识你～今天有什么想聊聊的话题吗？或者需要帮忙的地方？😊


让我们再试一个例子。系统消息是，你是一个友好的聊天机器人，第一个用户消息是，是的，你能提醒我我的名字是什么吗？

In [7]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},    
{'role':'user', 'content':'好，你能提醒我，我的名字是什么吗？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

哦，我现在还没有记住信息的能力呢，所以每次对话都是全新的开始～ 如果你愿意的话，可以直接告诉我你的名字，我会很开心用你喜欢的名字来称呼你！ 😊


如上所见，模型实际上并不知道我的名字。

因此，每次与语言模型的交互都<span style="color:red">互相独立</span>，这意味着我们必须提供所有相关的消息，以便模型在当前对话中进行引用。如果想让模型引用或 “记住” 对话的早期部分，则必须在模型的输入中提供早期的交流。我们将其称为<span style="color:red">上下文 (context)</span> 。尝试以下示例。

In [8]:
# 中文
messages =  [  
{'role':'system', 'content':'你是个友好的聊天机器人。'},
{'role':'user', 'content':'Hi, 我是Isa'},
{'role':'assistant', 'content': "Hi Isa! 很高兴认识你。今天有什么可以帮到你的吗?"},
{'role':'user', 'content':'是的，你可以提醒我, 我的名字是什么?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

你的名字是Isa呀～✨ 是不是想考考我的记忆力呀？哈哈，随时欢迎提问哦！😊


现在我们已经给模型提供了上下文，也就是之前的对话中提到的我的名字，然后我们会问同样的问题，也就是我的名字是什么。因为模型有了需要的全部上下文，所以它能够做出回应，就像我们在输入的消息列表中看到的一样。

## 三、订餐机器人

现在，我们构建一个 “订餐机器人”，我们需要它自动收集用户信息，接受比萨饼店的订单。

下面这个函数将收集我们的用户消息，以便我们可以避免像刚才一样手动输入。这个函数将从我们下面构建的用户界面中收集 Prompt ，然后将其附加到一个名为上下文( ```context``` )的列表中，并在每次调用模型时使用该上下文。模型的响应也会添加到上下文中，所以用户消息和模型消息都被添加到上下文中，上下文逐渐变长。这样，模型就有了需要的信息来确定下一步要做什么。

<span style="color:#00ff00">
原先的格式会报错。利用sonnet-3.7生成了一个美观的新背景。此外，采用deepseek-r1模型，无论如何优化prompt都很难实现我们每次只让模型生成一个回复的需求（r1会生成很多类似假设用户点了什么什么之类的内容），但是deepseek-v3模型可以实现。此外，gpt-4o-mini模型也可以实现。
</span>

<span style="color:#00ff00">
这里采用gpt-4o-mini模型作为示例。
</span>

In [7]:
# 以下是针对 Jupyter Notebook 的修复代码
# 复制这段代码并在您的 Notebook 中执行

import panel as pn
pn.extension()

# 定义自定义CSS样式 - 黑底白字主题
pn.config.raw_css.append('''
body {
    background-color: #121212 !important;
    color: #FFFFFF !important;
}
.dark-theme {
    background-color: #121212;
    color: #FFFFFF;
}
.dashboard-container {
    background-color: #121212;
    color: #FFFFFF;
    padding: 20px;
    border-radius: 10px;
}
.user-msg {
    background-color: #1E1E1E;
    color: #FFFFFF;
    padding: 10px;
    border-radius: 5px;
    margin-bottom: 10px;
}
.assistant-msg {
    background-color: #2D2D2D;
    color: #FFFFFF;
    padding: 10px;
    border-radius: 5px;
    margin-bottom: 10px;
    border-left: 3px solid #BB86FC;
}
.panel-widget-box {
    background-color: #121212 !important;
}
.bk-input {
    background-color: #2D2D2D !important;
    color: #FFFFFF !important;
    border: 1px solid #444444 !important;
}
.bk-btn {
    background-color: #BB86FC !important;
    color: #000000 !important;
    border: none !important;
}
.bk-btn:hover {
    background-color: #9969DA !important;
}
''')

# 修复后的collect_messages函数
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)  
    context.append({'role':'assistant', 'content':f"{response}"})
    
    # 用户消息使用user-msg类
    panels.append(
        pn.Row('用户:', pn.pane.Markdown(prompt, width=600, css_classes=['user-msg'])))
    
    # 助手消息使用assistant-msg类
    panels.append(
        pn.Row('助手:', pn.pane.Markdown(response, width=600, css_classes=['assistant-msg'])))
 
    return pn.Column(*panels, css_classes=['dashboard-container'])

# 运行后，使用此函数替换您Notebook中的collect_messages函数
# 然后重新运行dashboard的创建代码


现在，我们将设置并运行这个 UI 来显示订单机器人。初始的上下文包含了包含菜单的系统消息，在每次调用时都会使用。此后随着对话进行，上下文也会不断增长。运行结果可交互。

<span style="color:#00ff00">
同样利用sonnet-3.7自动延长输出高度，确保对话能够完整显示。
</span>

In [8]:
panels = [] # collect display 

context = [{'role':'system', 'content':"""
你是一个订餐机器人，为披萨餐厅自动收集订单信息。

收集订单信息步骤：
1.问候顾客。
2.收集订单信息。
3.确认顾客是否还需要添加其他内容。
4.询问是否自取或外送。
5.如果是外送，需要询问地址。
6.最后告诉顾客订单总金额，并送上祝福。

请注意，你可以多次回复，但是你的每次回复必须满足如下要求：

1.收集订单信息必须一步一步进行，不要跳过任何一步。
2.每完成收集订单信息的一步时，结束本次回复，等待顾客回复。
3.请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。
4.你的回应应该以简短、友好的风格呈现。
5.不要生成类似“（等待顾客回复）”等内容。
6.不要生成类似举例等内容，只按照要求生成回复。
菜单包括：

菜品：
意式辣香肠披萨（大、中、小） 12.95、10.00、7.00
芝士披萨（大、中、小） 10.95、9.25、6.50
茄子披萨（大、中、小） 11.95、9.75、6.75
薯条（大、小） 4.50、3.50
希腊沙拉 7.25

配料：
奶酪 2.00
蘑菇 1.50
香肠 3.00
加拿大熏肉 3.50
AI酱 1.50
辣椒 1.00

饮料：
可乐（大、中、小） 3.00、2.00、1.00
雪碧（大、中、小） 3.00、2.00、1.00
瓶装水 5.00
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

# 创建带有黑色背景的dashboard，让高度自动适应内容
dashboard = pn.Column(
    pn.pane.Markdown("# 披萨订餐机器人", css_classes=['dark-theme']),
    pn.layout.Divider(),
    # 聊天内容区域 - 没有固定高度限制
    pn.panel(interactive_conversation, loading_indicator=True),
    pn.layout.Divider(),
    pn.Row(
        inp,
        button_conversation
    ),
    css_classes=['dashboard-container'],
    width=800  # 只固定宽度，高度自适应
)


# 在Jupyter中使用此函数来增大显示区域的高度上限
def expand_output_height():
    from IPython.display import display, HTML
    display(HTML('''
    <style>
    /* 移除高度限制，让Jupyter输出区域自动扩展 */
    .jp-OutputArea-output {
        max-height: none !important;
    }
    .jp-Cell-outputArea {
        max-height: none !important;
    }
    .jp-OutputArea-child {
        overflow: visible !important;
    }
    </style>
    '''))

# 运行这个函数来扩展Jupyter的输出区域高度
expand_output_height()
    

In [9]:
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'a1c8de16-071e-4c88-b862-bb002bd09c53': {'version…

### 3.1 创建JSON摘要

此处我们另外要求模型创建一个 JSON 摘要，方便我们发送给订单系统。

因此我们需要在上下文的基础上追加另一个系统消息，作为另一条指示 (instruction) 。我们说*创建一个刚刚订单的 JSON 摘要，列出每个项目的价格，字段应包括 1）披萨，包括尺寸，2）配料列表，3）饮料列表，4）辅菜列表，包括尺寸，最后是总价格*。此处也可以定义为用户消息，不一定是系统消息。

请注意，这里我们使用了一个较低的温度，因为对于这些类型的任务，我们希望输出相对可预测。

In [10]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'创建上一个食品订单的 json 摘要。\
逐项列出每件商品的价格，字段应该是 1) 披萨，包括大小 2) 配料列表 3) 饮料列表，包括大小 4) 配菜列表包括大小 5) 总价'},    
)

response = get_completion_from_messages(messages,temperature=0)
print(response)

```json
{
  "订单": {
    "披萨": {
      "类型": "芝士披萨",
      "大小": "大",
      "价格": 10.95
    },
    "配料": [
      {
        "类型": "加拿大熏肉",
        "价格": 3.50
      }
    ],
    "饮料": [],
    "配菜": [],
    "总价": 14.45
  }
}
```


现在，我们已经建立了自己的订餐聊天机器人。请随意自定义并修改系统消息，以更改聊天机器人的行为，并使其扮演不同的角色，拥有不同的知识。

附：下图展示了订餐机器人一次完整的对话流程：
![image.png](../../figures/Chatbot-pizza-cn.png)